In [2]:
import xarray as xrs
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cfgrib
import math
import os
import FWI

ds = xrs.open_dataset("2015_abril_maio.grib", engine='cfgrib')
print(ds)

/home/ori/.local/lib/python3.10/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.24.2
  warnings.warn(


<xarray.Dataset> Size: 629kB
Dimensions:    (time: 61, latitude: 23, longitude: 16)
Coordinates:
  * time       (time) datetime64[ns] 488B 2015-04-01T12:00:00 ... 2015-05-31T...
    surface    float64 8B ...
  * latitude   (latitude) float64 184B 42.2 41.95 41.7 41.45 ... 37.2 36.95 36.7
  * longitude  (longitude) float64 128B 350.2 350.4 350.7 ... 353.4 353.7 353.9
Data variables:
    fbupinx    (time, latitude, longitude) float32 90kB ...
    drtcode    (time, latitude, longitude) float32 90kB ...
    dufmcode   (time, latitude, longitude) float32 90kB ...
    ffmcode    (time, latitude, longitude) float32 90kB ...
    fdsrte     (time, latitude, longitude) float32 90kB ...
    fwinx      (time, latitude, longitude) float32 90kB ...
    infsinx    (time, latitude, longitude) float32 90kB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Con

In [3]:
def DistanceTwoPoints(lat1, lon1, lat2, lon2):
    R = 6371e3  # Radius of the Earth in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

def find_nearest(lat, lon, latitudes, longitudes):
    min_distance = None
    nearest_point = None

    for lat2, lon2 in zip(latitudes, longitudes):
        distance = DistanceTwoPoints(lat, lon, lat2, lon2)
        if min_distance is None or distance < min_distance:
            min_distance = distance
            nearest_point = (lat2, lon2)

    return min_distance, nearest_point

def find_nearest_index(lat, lon, latitudes, longitudes):
    distances = np.array([DistanceTwoPoints(lat, lon, lat2, lon2) for lat2, lon2 in zip(latitudes, longitudes)])
    sorted_distances_indices = np.argsort(distances)
    nearest_points = [(latitudes[i], longitudes[i]) for i in sorted_distances_indices[:4]]
    min_distances = distances[sorted_distances_indices[:4]]
    return min_distances, nearest_points

def convert_longitude360(lon):
    if lon < 0:
        return lon + 360
    return lon

def convert_longitude(lon):
    if lon > 180:
        return lon - 360
    return lon


def total_precipitation(initial_date, final_date, dmet):
    dmet['hourly.time'] = pd.to_datetime(dmet['hourly.time'])
    dmet_copy = dmet.copy()
    dmet_copy.set_index('hourly.time', inplace=True)
    selected_rows = dmet_copy.loc[initial_date:final_date]
    total_precipitation = selected_rows['hourly.precipitation'].sum()

    return total_precipitation

In [5]:
fwinx_at_time = ds['fwinx'].sel(time='2015-04-29T12:00:00')

print(fwinx_at_time)

latitudes = fwinx_at_time.latitude
longitudes = fwinx_at_time.longitude

# Print the latitudes
print(latitudes.values)
print(len(latitudes.values))
print(longitudes.values)
print(len(longitudes.values))

novas_lat = []
novas_lon = []
for lat in range(len(latitudes)):
    for lon in range(len(longitudes)):
        novas_lat.append(latitudes[lat].values)
        novas_lon.append(longitudes[lon].values)

<xarray.DataArray 'fwinx' (latitude: 23, longitude: 16)> Size: 1kB
[368 values with dtype=float32]
Coordinates:
    time       datetime64[ns] 8B 2015-04-29T12:00:00
    surface    float64 8B ...
  * latitude   (latitude) float64 184B 42.2 41.95 41.7 41.45 ... 37.2 36.95 36.7
  * longitude  (longitude) float64 128B 350.2 350.4 350.7 ... 353.4 353.7 353.9
Attributes: (12/28)
    GRIB_paramId:                             260540
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      368
    GRIB_typeOfLevel:                         surface
    GRIB_stepType:                            instant
    GRIB_gridType:                            regular_ll
    ...                                       ...
    GRIB_name:                                Forest fire weather index
    GRIB_shortName:                           fwinx
    GRIB_units:                               Numeric
    long_name:                                Forest fire weather index
    uni

In [6]:
estacoes = [f for f in os.listdir("estacoes") if f.endswith('.csv')]

for est in estacoes:
    df = pd.read_csv("estacoes/" + est)
    LATITUDE_SING = df.iloc[0]['latitude']
    LONGITUDE_SING = df.iloc[0]['longitude']
    min_distance, nearest_point = find_nearest(LATITUDE_SING, convert_longitude360(LONGITUDE_SING), novas_lat, novas_lon)
    print(min_distance, nearest_point)

    print("Latitude: ", nearest_point[0])
    print("Longitude: ", nearest_point[1])

    fwinx_value = ds['fwinx'].sel(time=f'2015-04-29T12:00:00', latitude=nearest_point[0], longitude=nearest_point[1])
    drtcode_value = ds['drtcode'].sel(time=f'2015-04-29T12:00:00', latitude=nearest_point[0], longitude=nearest_point[1])
    dufmcode_value = ds['dufmcode'].sel(time=f'2015-04-29T12:00:00', latitude=nearest_point[0], longitude=nearest_point[1])
    ffmcode_value = ds['ffmcode'].sel(time=f'2015-04-29T12:00:00', latitude=nearest_point[0], longitude=nearest_point[1])

    dmet_filtered = df[df['hourly.time'].str.contains('T12:00')]
    #dmet_filtered.to_csv(f'estacoes12/MD_{est}', index=False)
    
    start_index = 0 + 1
    LAT = nearest_point[0]
    FFMCPrev = ffmcode_value.values.item()
    DMCPrev = dufmcode_value.values.item()
    DCPrev = drtcode_value.values.item()
    
    mts = {
        4: 30,
        5: 31,
        6: 30,
        7: 31,
        8:31, 
        9:30, 
        10:31}


    fwix_values = [None] * len(dmet_filtered)
    ffmc_values = [None] * len(dmet_filtered)
    dmc_values = [None] * len(dmet_filtered)
    dc_values = [None] * len(dmet_filtered)
    isi_values = [None] * len(dmet_filtered)
    bui_values = [None] * len(dmet_filtered)


    for i in range(start_index, len(dmet_filtered)):
        MONTH = (dmet_filtered.iloc[i]['hourly.time']).split('-')[1]
        DAY = (dmet_filtered.iloc[i]['hourly.time']).split('-')[2].split('T')[0]

        print(MONTH, DAY)

        TEMP = dmet_filtered.iloc[i]['hourly.temperature_2m']
        RH = dmet_filtered.iloc[i]['hourly.relative_humidity_2m']
        WIND = dmet_filtered.iloc[i]['hourly.wind_speed_10m']


        if(int(DAY) == 1):
            LAST_DAY = mts[int(MONTH) - 1]
            LAST_MONTH = str(int(MONTH) - 1)
            
            if(int(LAST_MONTH) < 10):
                LAST_MONTH = '0' + str(int(LAST_MONTH))
                
            initial_date = f'2015-{LAST_MONTH}-{LAST_DAY}T12:00'
            final_date = f'2015-{MONTH}-{DAY}T12:00'

        elif((int(DAY) < 10) and (int(DAY) > 1)):
            id = '0' + str(int(DAY)-1)
            fd = '0' + str(int(DAY))

            initial_date = f'2015-{MONTH}-{id}T12:00'
            final_date = f'2022-{MONTH}-{fd}T12:00'
        else:
            id = str(int(DAY)-1)
            fd = str(int(DAY))

            initial_date = f'2015-{MONTH}-{id}T12:00'
            final_date = f'2015-{MONTH}-{fd}T12:00'

    
        RAIN = total_precipitation(initial_date, final_date, df)

        FFMCPrev = FWI.FFMC(TEMP,RH,WIND,RAIN,FFMCPrev)
        DMCPrev = FWI.DMC(TEMP,RH,RAIN,DMCPrev,LAT,int(MONTH))
        DCPrev = FWI.DC(TEMP,RAIN,DCPrev,LAT,int(MONTH))
        isi = FWI.ISI(WIND, FFMCPrev)
        bui = FWI.BUI(DMCPrev,DCPrev)
        fwix = FWI.FWI(isi, bui)

        print("FFMC: ", FFMCPrev)
        print("DMC: ", DMCPrev)
        print("DC: ", DCPrev)
        print("ISI: ", isi)
        print("BUI: ", bui)
        print("FWI: ", fwix)

        fwix_values[i] = fwix
        ffmc_values[i] = FFMCPrev
        dmc_values[i] = DMCPrev
        dc_values[i] = DCPrev
        isi_values[i] = isi
        bui_values[i] = bui


    dmet_filtered['fwix'] = fwix_values
    dmet_filtered['ffmc'] = ffmc_values
    dmet_filtered['dmc'] = dmc_values
    dmet_filtered['dc'] = dc_values
    dmet_filtered['isi'] = isi_values
    dmet_filtered['bui'] = bui_values


    #dmet_filtered.to_csv(f'estacoes12X/fwi_{est}', index=False)

16179.920885674475 (array(38.45), array(350.95))
Latitude:  38.45
Longitude:  350.95
04 30
FFMC:  83.00468604989575
DMC:  12.5994163625
DC:  221.79537683105468
ISI:  1.976479725674276
BUI:  22.065214395477575
FWI:  3.3010223595112946
05 01
FFMC:  83.73626379748428
DMC:  14.3132780225
DC:  227.78137683105467
ISI:  2.882445281408991
BUI:  24.74003354536412
FWI:  5.3809050810390735
05 02
FFMC:  64.5883128665073
DMC:  11.758750014933788
DC:  228.86640817324258
ISI:  0.7722020165717152
BUI:  20.840615414392303
FWI:  0.7184725940534346
05 03
FFMC:  53.138608197410285
DMC:  9.111267040497554
DC:  229.74049846093024
ISI:  0.3454164397557756
BUI:  16.578790503979352
FWI:  0.27875294618302315
05 04
FFMC:  55.01499151412294
DMC:  6.775181241291468
DC:  230.730138311006
ISI:  1.655139932262262
BUI:  12.623656894668242
FWI:  1.5216908543252288
05 05
FFMC:  71.76425153157314
DMC:  8.623081713209169
DC:  236.338138311006
ISI:  1.1909809110806233
BUI:  15.804543866274177
FWI:  0.9336907537501528
05 06

/tmp/ipykernel_9756/3032285365.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_9756/3032285365.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_9756/3032285365.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

FFMC:  83.1571130154563
DMC:  8.678890396003318
DC:  44.214547760963164
ISI:  3.3868679568284596
BUI:  11.643844250833787
FWI:  3.9360395152646865
05 09
FFMC:  85.7694555144356
DMC:  11.244154300003318
DC:  50.07454776096316
ISI:  3.04696442979072
BUI:  14.402926912974735
FWI:  4.01030297929169
05 10
FFMC:  88.81167773096769
DMC:  15.000433588003318
DC:  56.708547760963164
ISI:  7.102074506830115
BUI:  18.05872263654736
FWI:  10.146020889639884
05 11
FFMC:  88.86064288259564
DMC:  18.505030580003318
DC:  63.522547760963164
ISI:  7.0448684170968825
BUI:  21.41431632536007
FWI:  11.010541156131199
05 12
FFMC:  90.71257684125075
DMC:  23.071116084003318
DC:  70.60654776096317
ISI:  10.57779914716291
BUI:  25.396270276618083
FWI:  16.50112705865536
05 13
FFMC:  91.83938032555365
DMC:  27.880985904003317
DC:  77.51054776096316
ISI:  12.478794306565863
BUI:  29.35977536941079
FWI:  20.010471542722698
05 14
FFMC:  87.91513030192884
DMC:  29.75912554800332
DC:  82.84854776096316
ISI:  7.337728

/tmp/ipykernel_9756/3032285365.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_9756/3032285365.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_9756/3032285365.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

FFMC:  90.29089230803598
DMC:  62.08197416989628
DC:  128.452
ISI:  5.9267462092576455
BUI:  61.94280173358984
FWI:  16.99984502195482
05 27
FFMC:  91.93320247401257
DMC:  66.63673923589627
DC:  134.924
ISI:  10.871449332035414
BUI:  66.47429241565669
FWI:  27.220220065985995
05 28
FFMC:  91.9831981981126
DMC:  71.31760871589627
DC:  141.702
ISI:  6.956677015911251
BUI:  71.13197599002723
FWI:  20.591437312350386
05 29
FFMC:  92.03321042831818
DMC:  75.86473906789627
DC:  148.48
ISI:  9.43167867397249
BUI:  75.65754254295774
FWI:  26.352406611828574
05 30
FFMC:  92.01285940649088
DMC:  79.85190263789627
DC:  154.88
ISI:  9.940586835256493
BUI:  79.62817764953542
FWI:  28.040213648822576
05 31
FFMC:  90.8512779281957
DMC:  83.33333222189627
DC:  161.262
ISI:  8.909039511992697
BUI:  83.09943785699494
FWI:  26.570321260258485
13890.007071131957 (array(41.95), array(351.45))
Latitude:  41.95
Longitude:  351.45
04 30
FFMC:  63.09040784653973
DMC:  2.3639260382824903
DC:  11.866445529937744

...

/tmp/ipykernel_9756/3032285365.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_9756/3032285365.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_9756/3032285365.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

FFMC:  93.59385750810492
DMC:  56.733901454279035
DC:  261.78308496148543
ISI:  10.615062948915579
BUI:  73.59424844165314
FWI:  28.183497369183662
05 19
FFMC:  92.68623941484053
DMC:  60.91430226827904
DC:  268.2550849614854
ISI:  18.182869405544803
BUI:  77.7121732375987
FWI:  41.34482380948224
05 20
FFMC:  92.73648375434666
DMC:  65.06548055627904
DC:  274.3490849614854
ISI:  12.80414377086753
BUI:  81.69397148397786
FWI:  33.666652239114036
05 21
FFMC:  92.78674468203883
DMC:  69.27352430027904
DC:  280.5510849614854
ISI:  12.077191123078942
BUI:  85.66571619334799
FWI:  33.15589110384438
05 22
FFMC:  93.04622052426824
DMC:  73.83223835627903
DC:  286.7530849614854
ISI:  13.37250416876627
BUI:  89.83708560778386
FWI:  36.29787810359126
05 23
FFMC:  93.09658371166196
DMC:  78.34856658627903
DC:  293.1890849614854
ISI:  11.403984910183688
BUI:  93.93907084477702
FWI:  33.38274767452063
05 24
FFMC:  93.14696352647941
DMC:  82.92807865627904
DC:  299.80508496148536
ISI:  12.13919779792

/tmp/ipykernel_9756/3032285365.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_9756/3032285365.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_9756/3032285365.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

FFMC:  91.75658698334664
DMC:  73.13591268214266
DC:  138.84800000000004
ISI:  11.32078902303989
BUI:  72.90970828124607
FWI:  29.308389960085574
05 28
FFMC:  92.20062923007875
DMC:  78.10795435814266
DC:  145.84200000000004
ISI:  8.014718375195505
BUI:  77.85568579917927
FWI:  23.877328456184927
05 29
FFMC:  92.25071324521464
DMC:  82.78197892214266
DC:  152.89000000000004
ISI:  8.112339788113433
BUI:  82.50759969871862
FWI:  24.806564996177865
05 30
FFMC:  92.30081379560544
DMC:  87.17404560014266
DC:  159.57800000000003
ISI:  9.647898637064198
BUI:  86.87887614044386
FWI:  28.70532364988564
05 31
FFMC:  91.15508192925245
DMC:  90.80816946414267
DC:  166.06800000000004
ISI:  8.243446565537635
BUI:  90.50105734189027
FWI:  26.30663576694964
8475.22598205075 (array(40.45), array(352.45))
Latitude:  40.45
Longitude:  352.45
04 30
FFMC:  81.87618410849302
DMC:  8.857024984
DC:  58.32269696044922
ISI:  2.8722724758642593
BUI:  12.83946855658011
FWI:  3.4758419335467496
05 01
FFMC:  72.411

/tmp/ipykernel_9756/3032285365.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['fwix'] = fwix_values
/tmp/ipykernel_9756/3032285365.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmet_filtered['ffmc'] = ffmc_values
/tmp/ipykernel_9756/3032285365.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable